In [5]:
from fcn import TwoLayerFCN
from fcn_edge import TwoLayerFCNEdge
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
data_name = 'Cora'

### Node classification with FCN

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = Planetoid('./data', data_name, transform=T.NormalizeFeatures())
data = dataset[0]

In [4]:
num_nodes, num_features = data.x.size()
num_classes = len(data.y.unique())

In [5]:
torch.manual_seed(42)

y = F.one_hot(data.y)
y_tr, y_val, y_te = y[data.train_mask], y[data.val_mask], y[data.test_mask]
y_tr = torch.cat([y_tr, torch.zeros(size=(num_nodes - y_tr.shape[0], num_classes))], axis=0)
y_val = torch.cat([y_val, torch.zeros(size=(num_nodes - y_val.shape[0], num_classes))], axis=0)
y_te = torch.cat([y_te, torch.zeros(size=(num_nodes - y_te.shape[0], num_classes))], axis=0)

X = data.x
X_tr, X_val, X_te = X[data.train_mask], X[data.val_mask], X[data.test_mask]
X_tr = torch.cat([X_tr, torch.zeros(size=(num_nodes - X_tr.shape[0], num_features))], axis=0)
X_val = torch.cat([X_val, torch.zeros(size=(num_nodes - X_val.shape[0], num_features))], axis=0)
X_te = torch.cat([X_te, torch.zeros(size=(num_nodes - X_te.shape[0], num_features))], axis=0)

tr_shuffle, val_shuffle, te_shuffle = torch.randperm(num_nodes), torch.randperm(num_nodes), torch.randperm(num_nodes)
X_tr, X_val, X_te = X_tr[tr_shuffle], X_val[val_shuffle], X_te[te_shuffle]

In [6]:
lr = 1e-2
epochs = 300
print_every = 20

model = TwoLayerFCN(num_classes, num_features, num_nodes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

for t in range(1, epochs + 1):
    shuffle = torch.randperm(num_nodes)
    shuffle_mask = data.train_mask[shuffle]
    
    inputs, train_targets = X_tr[shuffle].to(device), y_tr[shuffle].to(device)
    train_preds = model(inputs)
    train_loss = loss_fn(train_preds[shuffle_mask], train_targets[shuffle_mask])
    train_acc = (train_preds.argmax(axis=1) == train_targets.argmax(axis=1)).float().mean()

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    with torch.no_grad():
        shuffle = torch.randperm(num_nodes)
        shuffle_mask = data.val_mask[shuffle]

        inputs, val_targets = X_val[shuffle].to(device), y_val[shuffle].to(device)
        val_preds = model(inputs)
        val_loss = loss_fn(val_preds[shuffle_mask], val_targets[shuffle_mask])
        val_acc = (val_preds.argmax(axis=1) == val_targets.argmax(axis=1)).float().mean()

    if t % print_every == 0 or t == 1: print(
        'Epoch {:3d}'.format(t) 
        + ' | tr xe: {:.6f}; val xe: {:.6f};'.format(train_loss.item(), val_loss.item())
        + ' tr acc: {:.6f}; val acc: {:.6f}'.format(train_acc.item(), val_acc.item())
    )

Epoch   1 | Training CE: 1.945910; Validation CE: 1.401050; Training ACC: 0.151773; Validation ACC: 0.147341
Epoch  20 | Training CE: 1.915074; Validation CE: 1.407285; Training ACC: 0.287297; Validation ACC: 0.292097
Epoch  40 | Training CE: 1.911815; Validation CE: 1.407049; Training ACC: 0.529911; Validation ACC: 0.471935
Epoch  60 | Training CE: 1.920423; Validation CE: 1.403476; Training ACC: 0.753323; Validation ACC: 0.669498
Epoch  80 | Training CE: 1.906999; Validation CE: 1.404551; Training ACC: 0.825702; Validation ACC: 0.719719
Epoch 100 | Training CE: 1.911252; Validation CE: 1.402851; Training ACC: 0.865583; Validation ACC: 0.756647
Epoch 120 | Training CE: 1.912264; Validation CE: 1.402810; Training ACC: 0.888109; Validation ACC: 0.773264
Epoch 140 | Training CE: 1.916586; Validation CE: 1.402214; Training ACC: 0.888479; Validation ACC: 0.769202
Epoch 160 | Training CE: 1.913640; Validation CE: 1.403581; Training ACC: 0.916544; Validation ACC: 0.797267
Epoch 180 | Trainin

### Link prediction with FCN

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = Planetoid('./data', data_name, transform=T.NormalizeFeatures())
data = dataset[0]

In [4]:
num_nodes, num_features = data.x.size()

In [5]:
X = data.x
A = torch.sparse_coo_tensor(data.edge_index, torch.ones(data.edge_index.shape[1])).to_dense()

# separate into train, val, test
X_tr, X_val, X_te = X[data.train_mask], X[data.val_mask], X[data.test_mask]
A_tr, A_val, A_te = A[data.train_mask], A[data.val_mask], A[data.test_mask]

# get indices of train, val, test
mask_tr = torch.cat([torch.ones(len(X_tr)), torch.zeros(num_nodes - len(X_tr))]).bool()
mask_val = torch.cat([torch.ones(len(X_val)), torch.zeros(num_nodes - len(X_val))]).bool()
mask_te = torch.cat([torch.ones(len(X_te)), torch.zeros(num_nodes - len(X_te))]).bool()

# fix shapes
X_tr = torch.cat([X_tr, torch.zeros(size=(num_nodes - X_tr.shape[0], num_features))], axis=0)
A_tr = torch.cat([A_tr, torch.zeros(size=(num_nodes - A_tr.shape[0], num_nodes))], axis=0)

X_val = torch.cat([X_val, torch.zeros(size=(num_nodes - X_val.shape[0], num_features))], axis=0)
A_val = torch.cat([A_val, torch.zeros(size=(num_nodes - A_val.shape[0], num_nodes))], axis=0)

X_te = torch.cat([X_te, torch.zeros(size=(num_nodes - X_te.shape[0], num_features))], axis=0)
A_te = torch.cat([A_te, torch.zeros(size=(num_nodes - A_te.shape[0], num_nodes))], axis=0)

# permute
shuffle_tr, shuffle_val, shuffle_te = torch.randperm(num_nodes), torch.randperm(num_nodes), torch.randperm(num_nodes)
X_tr, X_val, X_te = X_tr[shuffle_tr], X_val[shuffle_val], X_te[shuffle_te]
A_tr, A_val, A_te = A_tr[shuffle_tr], A_val[shuffle_val], A_te[shuffle_te]
mask_tr, mask_val, mask_te = mask_tr[shuffle_tr], mask_val[shuffle_val], mask_te[shuffle_te]

In [6]:
def report(targets, preds, mask):
    edge_rows, edge_cols = targets[mask].nonzero().t()
    tp = len(targets[mask].nonzero()[:, 0])
    fp = len((targets[mask][~edge_rows, ~edge_cols] != preds[mask][~edge_rows, ~edge_cols].round()).nonzero())
    fn = len((targets[mask][edge_rows, edge_cols] != preds[mask][edge_rows, edge_cols].round()).nonzero())
    prec, recall = tp / (tp + fp), tp / (tp + fn)
    return prec, recall

In [7]:
lr = 1e-4
epochs = 600
print_every = 15

model = TwoLayerFCNEdge(num_nodes, num_features, num_features // 4, num_features // 16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

for t in range(1, epochs + 1):
    model.train()
    shuffle = torch.randperm(num_nodes)
    input, targets, shuffle_mask = X_tr[shuffle].to(device), A_tr[shuffle].to(device), mask_tr[shuffle]
    preds = model(input)
    train_loss = loss_fn(preds[mask_tr], targets[mask_tr])

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    with torch.no_grad():
        model.eval()
        shuffle = torch.randperm(num_nodes)

        inputs, targets, shuffle_mask = X_tr[shuffle].to(device), A_tr[shuffle].to(device), mask_tr[shuffle]
        preds = model(inputs)
        tr_prec, tr_rcll = report(targets, preds, shuffle_mask)

        input, targets, shuffle_mask = X_val[shuffle].to(device), A_val[shuffle].to(device), mask_val[shuffle]
        preds = model(input)
        val_loss = loss_fn(preds[shuffle_mask], targets[shuffle_mask])
        val_prec, val_rcll = report(targets, preds, shuffle_mask)

    if t % print_every == 0 or t == 1: print(
        'Epoch {:3d} | '.format(t) 
        + 'tr xe: {:.4f}; val xe: {:.4f}; '.format(train_loss.item(), val_loss.item())
        + 'tr recall: {:.4f}; val recall: {:.4f}; '.format(tr_rcll, tr_rcll)
        + 'tr prec: {:.4f}; val prec: {:.4f}'.format(tr_prec, val_prec) 
    )
with torch.no_grad():
    model.eval()
    inputs, targets = X_te.to(device), A_te.to(device)
    preds = model(inputs)
    test_prec, test_rcll = report(targets, preds, mask_te)
    print('test recall: {:.4f}; test prec: {:.4f}'.format(test_rcll, test_prec))

Epoch   1 | tr xe: 0.5081; val xe: 30.7939; tr recall: 1.0000; val recall: 1.0000; tr prec: 0.5008; val prec: 0.5005
Epoch  15 | tr xe: 1.2421; val xe: 30.7938; tr recall: 1.0000; val recall: 1.0000; tr prec: 0.5016; val prec: 0.5008
Epoch  30 | tr xe: 1.4679; val xe: 30.7938; tr recall: 1.0000; val recall: 1.0000; tr prec: 0.5000; val prec: 0.5005
Epoch  45 | tr xe: 3.7261; val xe: 30.7938; tr recall: 1.0000; val recall: 1.0000; tr prec: 0.5000; val prec: 0.5005
Epoch  60 | tr xe: 2.2583; val xe: 30.7938; tr recall: 1.0000; val recall: 1.0000; tr prec: 0.5016; val prec: 0.5000
Epoch  75 | tr xe: 1.2420; val xe: 30.7938; tr recall: 1.0000; val recall: 1.0000; tr prec: 0.5024; val prec: 0.5013
Epoch  90 | tr xe: 1.9759; val xe: 30.7938; tr recall: 1.0000; val recall: 1.0000; tr prec: 0.5004; val prec: 0.5003
Epoch 105 | tr xe: 0.7339; val xe: 30.7937; tr recall: 1.0000; val recall: 1.0000; tr prec: 0.5000; val prec: 0.5003
Epoch 120 | tr xe: 2.5405; val xe: 30.7937; tr recall: 1.0000; v

In [16]:
# X, y = [], []
# existing_edges = [data.edge_index[:, i] for i in range(data.edge_index.shape[1])]
# for e in tqdm(existing_edges):
#     n1, n2 = e
#     # positive sample
#     X.append((data.x[n1] - data.x[n2]).abs())
#     y.append(torch.Tensor([1,0]))

#     # negative sample
#     not_n1_neighbors = data.edge_index[1, data.edge_index[0] != n1]
#     random_idx = np.random.choice(len(not_n1_neighbors))
#     X.append((data.x[n1] - data.x[not_n1_neighbors[random_idx]]).abs())
#     y.append(torch.Tensor([0,1]))

# num_samples = len(X)
# tr, val, te = int(0.7 * num_samples), int(0.15 * num_samples), int(0.15 * num_samples)
# X_tr, X_val, X_te = X[:tr], X[tr:(tr + val)], X[(tr + val):]
# y_tr, y_val, y_te = y[:tr], y[tr:(tr + val)], y[(tr + val):]

100%|██████████| 10556/10556 [00:01<00:00, 7764.61it/s]


In [17]:
# batch_size = 32
# trainDataLoader = DataLoader(
#     [(input, target) for input, target in zip(X_tr, y_tr)],
#     shuffle=True,
#     batch_size=batch_size
# )
# valDataLoader = DataLoader(
#     [(input, target) for input, target in zip(X_val, y_val)],
#     shuffle=True,
#     batch_size=len(X_val)
# )
# testDataLoader = DataLoader(
#     [(input, target) for input, target in zip(X_te, y_te)],
#     shuffle=True,
#     batch_size=len(X_te)
# )

In [29]:
# lr = 1e-3
# epochs = 75
# print_every = 5

# model = TwoLayerFCNEdge(num_features, num_features // 2, num_features // 4).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# loss_fn = nn.CrossEntropyLoss()

# for t in range(1, epochs + 1):
#     for X, y in trainDataLoader:
#         inputs, targets = X.to(device), y.to(device)
#         train_preds = model(inputs)
#         train_loss = loss_fn(train_preds, targets)
#         train_acc = (train_preds.argmax(axis=1) == targets.argmax(axis=1)).float().mean()

#         optimizer.zero_grad()
#         train_loss.backward()
#         optimizer.step()

#     with torch.no_grad():
#         for X_sg, y_sg in valDataLoader: # has length of 1 because why not
#             X_sg, y_sg = X_sg.to(device), y_sg.to(device)
#             val_preds = model(X_sg)
#             val_loss = loss_fn(val_preds, y_sg)
#             val_acc = (val_preds.argmax(axis=1) == y_sg.argmax(axis=1)).float().mean()

#     if t % print_every == 0 or t == 1: print(
#         'Epoch {:3d} | '.format(t) 
#         + 'Training CE: {:.6f}; Validation CE: {:.6f}; '.format(train_loss.item(), val_loss.item())
#         + 'Training ACC: {:.6f}; Validation ACC: {:.6f}'.format(train_acc.item(), val_acc.item())
#     )

Epoch   1 | Training CE: 0.624491; Validation CE: 0.645586; Training ACC: 0.692308; Validation ACC: 0.639292
Epoch   5 | Training CE: 0.467465; Validation CE: 0.567438; Training ACC: 0.846154; Validation ACC: 0.728996
Epoch  10 | Training CE: 0.469128; Validation CE: 0.522194; Training ACC: 0.846154; Validation ACC: 0.783007
Epoch  15 | Training CE: 0.390870; Validation CE: 0.502896; Training ACC: 0.923077; Validation ACC: 0.808275
Epoch  20 | Training CE: 0.390542; Validation CE: 0.515589; Training ACC: 0.923077; Validation ACC: 0.790903
Epoch  25 | Training CE: 0.474843; Validation CE: 0.520048; Training ACC: 0.846154; Validation ACC: 0.787429
Epoch  30 | Training CE: 0.391237; Validation CE: 0.526192; Training ACC: 0.923077; Validation ACC: 0.783955
Epoch  35 | Training CE: 0.313303; Validation CE: 0.496157; Training ACC: 1.000000; Validation ACC: 0.814277
Epoch  40 | Training CE: 0.313309; Validation CE: 0.514026; Training ACC: 1.000000; Validation ACC: 0.796589
Epoch  45 | Trainin